<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=e10aee5a2be978f92ec9ecd2500f411298c121365cf3c7798823aad4efd51d1a
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65


   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-12 09:20:21
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.18 L
Current:  1.42 C
-------------------
Today PnL: 24.03 K (0.17%)
Current PnL: -17.52 L (-11.78%)
CY Booked + Current PnL: -6.49 L (-4.36%)
-------------------
Total profit:  2.08 L
Total loss:  -19.60 L
-------------------
Total Booked + Current PnL: 20.49 L (16.84%)
Total Booked PnL: 38.01 L (31.23%)
Curr Year Booked PnL: 11.03 L (7.78%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 83.62 L (59.02%)
Deployed:  1.22 C
Current:  1.42 C
CAGR/XIRR %: 9.5%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,APOLLOHOSP,8285.00,-12.98,61.0,H-LC,11.20,173899.0,17722.0,8365.0,0.32,11.35,4.81,16.71,22.0,2.12,1.26,30.76,X40N,BTT,HEALTHCARE
62,SBIN,863.00,-12.30,59.0,M-LC,4.26,217896.0,16416.0,10808.0,-0.17,8.15,4.96,13.51,60.0,1.52,1.58,21.77,XY25,NTT,BANKS
77,TTKPRESTIG,770.00,104.48,59.0,M-SC,4.98,88078.0,-12699.0,12789.0,-0.12,-12.60,14.52,0.09,245.0,-0.99,0.64,15.71,OX40N,NTT,DURABLES
49,NATIONALUM,247.44,-42.83,72.0,H-MC,4.25,114872.0,14368.0,16266.0,1.09,14.30,14.16,30.49,79.0,0.88,0.84,51.33,MH,ATH,METALS
41,ITC,452.00,-37.50,55.0,H-LC,2.98,202519.0,2381.0,18510.0,-0.22,1.19,9.14,10.44,4.0,0.13,1.47,6.82,X40,NTT,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,43.54,64.0,M-MC,7.00,226994.0,2032.0,165615.0,1.62,0.90,72.96,74.53,192.0,0.01,1.65,33.45,XY24,BTT,STEEL
35,ICICIPRULI,790.00,-21.22,41.0,H-MC,2.08,135611.0,-177.0,42934.0,0.27,-0.13,31.66,31.48,107.0,-0.00,0.99,11.97,X40,ATH,INSURANCE
51,PGHH,17973.08,-31.21,49.0,H-MC,4.54,199050.0,-1770.0,70543.0,-0.26,-0.88,35.44,34.25,80.0,-0.03,1.45,4.27,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-16.13,39.0,H-LC,6.03,302316.0,-13526.0,126187.0,-0.17,-4.28,41.74,35.67,5.0,-0.11,2.20,8.98,X40N,ATH,FMCG
43,JIOFIN,387.00,-1.56,43.0,H-LC,13.52,267546.0,181.0,65281.0,0.13,0.07,24.40,24.48,48.0,0.00,1.95,55.08,XY24,BTT,FINANCE
80,UNITDSPR,1644.00,-13.66,49.0,H-LC,8.03,226421.0,-7723.0,56356.0,-0.20,-3.30,24.89,20.77,86.0,-0.14,1.65,3.39,X40N,NTT,BREWERIES
31,HCLTECH,1943.91,1.08,50.0,H-LC,10.10,231292.0,-10632.0,73898.0,0.37,-4.39,31.95,26.15,8.0,-0.14,1.68,9.30,X40,ATH,IT
56,RELIANCE,1533.00,-13.66,50.0,H-LC,6.03,216294.0,5928.0,22862.0,0.23,2.82,10.57,13.68,37.0,0.26,1.57,19.81,XY25,NTT,REFINERIES


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.20,-3.76,58.0,H-LC,1.19,238455.0,-7696.0,86893.0,0.14,-3.13,36.44,32.18,16.0,-0.09,1.73,31.79,X200,ATH,IT
41,ITC,452.00,-37.50,55.0,H-LC,2.98,202519.0,2381.0,18510.0,-0.22,1.19,9.14,10.44,4.0,0.13,1.47,6.82,X40,NTT,FMCG
50,NESTLEIND,1377.00,-7.33,62.0,H-LC,3.56,285340.0,19914.0,36866.0,-0.11,7.50,12.92,21.40,11.0,0.54,2.08,15.38,XY25,NTT,FMCG
20,CIPLA,1795.00,-19.44,55.0,H-LC,5.89,215008.0,10508.0,30897.0,0.55,5.14,14.37,20.25,10.0,0.34,1.56,14.32,X40N,BTT,PHARMA
83,VBL,671.64,-16.13,39.0,H-LC,6.03,302316.0,-13526.0,126187.0,-0.17,-4.28,41.74,35.67,5.0,-0.11,2.20,8.98,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.0,3743.24,62.0,M-SC,7.27,119817.0,-27551.0,79367.0,4.77,-18.70,66.24,35.16,236.0,-0.35,0.87,22.38,XY24,NTT,AUTO
59,SAIL,228.0,43.54,64.0,M-MC,7.00,226994.0,2032.0,165615.0,1.62,0.90,72.96,74.53,192.0,0.01,1.65,33.45,XY24,BTT,STEEL
27,GREENPANEL,537.0,257.24,79.0,M-SC,5.20,159454.0,-23624.0,102609.0,1.43,-12.90,64.35,43.14,230.0,-0.23,1.16,49.99,XY24,NTT,MISC
39,INFY,2275.0,-16.37,57.0,H-LC,9.44,326146.0,13258.0,158442.0,1.42,4.24,48.58,54.87,3.0,0.08,2.37,11.17,X40,BTT,IT
4,ANGELONE,3033.0,4.08,35.0,H-SC,7.76,184090.0,-6916.0,64616.0,1.31,-3.62,35.10,30.21,157.0,-0.11,1.34,16.50,X40N,NTT,FINANCE


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SURYODAY,240.0,40.35,50.0,H-SC,3.27,128948.0,-50123.0,115847.0,-1.66,-27.99,89.84,36.71,135.0,-0.43,0.94,27.94,XR,NTT,BANKS
53,RAJESHEXPO,518.0,1845.71,65.0,L-SC,5.50,53080.0,-84097.0,84190.0,-1.08,-61.31,158.61,0.07,267.0,-1.00,0.39,32.01,OX40N,NTT,JEWELLERY
24,DABUR,735.0,-3.48,56.0,H-MC,6.17,210093.0,13065.0,76558.0,-0.99,6.63,36.44,45.49,102.0,0.17,1.53,20.54,XY24,BTT,FMCG
81,VAIBHAVGBL,521.0,50.41,57.0,H-SC,9.88,130419.0,-52356.0,163428.0,-0.81,-28.65,125.31,60.77,125.0,-0.32,0.95,16.81,XR,NTT,JEWELLERY
78,UJJIVANSFB,60.0,100.70,62.0,H-SC,12.58,126549.0,-15930.0,35446.0,-0.78,-11.18,28.01,13.70,163.0,-0.45,0.92,49.36,OX40N,NTT,BANKS


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,100.70,62.0,H-SC,12.58,126549.0,-15930.0,35446.0,-0.78,-11.18,28.01,13.70,163.0,-0.45,0.92,49.36,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,104.48,59.0,M-SC,4.98,88078.0,-12699.0,12789.0,-0.12,-12.60,14.52,0.09,245.0,-0.99,0.64,15.71,OX40N,NTT,DURABLES
19,CERA,9475.0,-19.81,47.0,H-SC,2.13,146176.0,-29727.0,71743.0,0.22,-16.90,49.08,23.89,149.0,-0.41,1.06,26.24,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-19.34,55.0,H-MC,7.92,109146.0,-22689.0,64909.0,-0.14,-17.21,59.47,32.03,98.0,-0.35,0.79,22.25,OX40N,NTT,IT
44,KANSAINER,340.0,-67.79,57.0,H-SC,2.05,221769.0,-47898.0,84228.0,-0.08,-17.76,37.98,13.47,138.0,-0.57,1.61,12.17,XY24,NTT,PAINTS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4998.00,-26.12,55.0,H-LC,13.40,294455.0,-51501.0,175348.0,0.27,-14.89,59.55,35.80,1.0,-0.29,2.14,4.31,X40,BTT,IT
39,INFY,2275.00,-16.37,57.0,H-LC,9.44,326146.0,13258.0,158442.0,1.42,4.24,48.58,54.87,3.0,0.08,2.37,11.17,X40,BTT,IT
41,ITC,452.00,-37.50,55.0,H-LC,2.98,202519.0,2381.0,18510.0,-0.22,1.19,9.14,10.44,4.0,0.13,1.47,6.82,X40,NTT,FMCG
83,VBL,671.64,-16.13,39.0,H-LC,6.03,302316.0,-13526.0,126187.0,-0.17,-4.28,41.74,35.67,5.0,-0.11,2.20,8.98,X40N,ATH,FMCG
1,ABB,7934.00,-40.25,52.0,H-LC,11.98,249758.0,-11861.0,131073.0,-0.28,-4.53,52.48,45.57,7.0,-0.09,1.82,6.27,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,43.54,64.0,M-MC,7.00,226994.0,2032.0,165615.0,1.62,0.90,72.96,74.53,192.0,0.01,1.65,33.45,XY24,BTT,STEEL
15,BLUESTARCO,2326.38,9.32,53.0,H-MC,3.96,188820.0,24150.0,43825.0,0.07,14.67,23.21,41.28,89.0,0.55,1.37,23.82,X40N,ATH,AC
84,VOLTAS,1918.49,-0.10,55.0,H-MC,2.98,209565.0,17823.0,78210.0,0.13,9.30,37.32,50.08,99.0,0.23,1.52,16.72,XY25,ATH,AC
24,DABUR,735.00,-3.48,56.0,H-MC,6.17,210093.0,13065.0,76558.0,-0.99,6.63,36.44,45.49,102.0,0.17,1.53,20.54,XY24,BTT,FMCG
18,CAMS,5211.76,-4.94,51.0,H-SC,4.72,108128.0,6124.0,37802.0,0.29,6.00,34.96,43.06,122.0,0.16,0.79,24.15,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,247.44,-42.83,72.0,H-MC,4.25,114872.0,14368.0,16266.0,1.09,14.30,14.16,30.49,79.0,0.88,0.84,51.33,MH,ATH,METALS
36,INDIAMART,4850.92,-50.53,54.0,H-SC,12.25,138028.0,14692.0,119077.0,-0.31,11.91,86.27,108.45,119.0,0.12,1.00,36.93,AR,ATH,MISC
85,WHIRLPOOL,2270.00,-41.17,49.0,M-SC,1.50,99375.0,7876.0,70874.0,-0.38,8.61,71.32,86.07,223.0,0.11,0.72,44.31,XR,NTT,DURABLES
86,WIPRO,420.00,-10.62,56.0,M-LC,6.13,157213.0,6268.0,103179.0,-0.13,4.15,65.63,72.51,53.0,0.06,1.14,10.43,XR,NTT,IT
12,BANKINDIA,190.00,-29.19,66.0,H-MC,5.69,182930.0,3122.0,109282.0,0.71,1.74,59.74,62.52,88.0,0.03,1.33,36.41,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,6694.44,50.0,L-SC,5.90,74578.0,-19232.0,95848.0,0.25,-20.50,128.52,81.67,269.0,-0.20,0.54,44.42,XR,NTT,CERAMICS
3,ALKYLAMINE,4546.37,-4.84,45.0,H-SC,14.02,88614.0,-12349.0,106877.0,0.53,-12.23,120.61,93.63,148.0,-0.12,0.64,31.99,SR,ATH,CHEMICALS
22,COFFEEDAY,80.00,-39.64,57.0,L-SC,44.64,88282.0,-25267.0,65320.0,0.68,-22.25,73.99,35.27,268.0,-0.39,0.64,116.07,XR,NTT,HOTELS
48,MASFIN,398.61,-16.44,57.0,H-SC,13.13,94950.0,-3030.0,24630.0,0.19,-3.09,25.94,22.05,152.0,-0.12,0.69,38.02,XR,ATH,FINANCE
49,NATIONALUM,247.44,-42.83,72.0,H-MC,4.25,114872.0,14368.0,16266.0,1.09,14.30,14.16,30.49,79.0,0.88,0.84,51.33,MH,ATH,METALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-39.64,57.0,L-SC,44.64,88282.0,-25267.0,65320.0,0.68,-22.25,73.99,35.27,268.0,-0.39,0.64,116.07,XR,NTT,HOTELS
8,ATULAUTO,844.00,3743.24,62.0,M-SC,7.27,119817.0,-27551.0,79367.0,4.77,-18.70,66.24,35.16,236.0,-0.35,0.87,22.38,XY24,NTT,AUTO
27,GREENPANEL,537.00,257.24,79.0,M-SC,5.20,159454.0,-23624.0,102609.0,1.43,-12.90,64.35,43.14,230.0,-0.23,1.16,49.99,XY24,NTT,MISC
49,NATIONALUM,247.44,-42.83,72.0,H-MC,4.25,114872.0,14368.0,16266.0,1.09,14.30,14.16,30.49,79.0,0.88,0.84,51.33,MH,ATH,METALS
60,SAMMAANCAP,326.00,-171.59,64.0,M-SC,2.99,82356.0,-19854.0,113248.0,0.24,-19.42,137.51,91.37,208.0,-0.18,0.60,33.69,XY25,NTT,FINANCE


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.76
1,25,43.48
2,50,73.87


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.50
LC    34.46
MC    23.02
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.97
X40      15.27
X40N     12.99
XR       10.04
XY25      9.75
AR        8.60
OX40N     8.22
X200      1.73
MH        1.72
X5K       1.42
SR        1.27
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    27.35
H-LC    26.28
H-MC    20.08
M-SC    13.58
M-LC     7.12
M-MC     2.63
L-SC     1.57
L-LC     1.06
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.40,-13.98,78.79
FMCG,11.88,-3.73,40.02
FINANCE,9.63,-16.31,61.86
BANKS,7.80,-11.75,63.19
MISC,6.12,-12.83,77.24
PAINTS,5.92,-13.54,38.95
ELECTRICAL,5.57,-7.58,66.86
AC,3.60,2.35,30.72
INSURANCE,3.38,-8.23,45.35


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2861315.0,22
AR,1314031.0,9
XR,1231523.0,13
X40,866033.0,10
X40N,626319.0,10
OX40N,517256.0,10
XY25,410503.0,7
SR,246329.0,2
X5K,122774.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3354017.0,28
M-SC,1378517.0,17
H-LC,1377994.0,15
H-MC,1192757.0,15
M-LC,399018.0,5
M-MC,315812.0,2
L-SC,245358.0,3
L-MC,59740.0,1
L-LC,38655.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1040799.0      6
M-SC       XY24       792226.0      7
H-SC       AR         782822.0      5
           XR         680202.0      6
H-LC       X40        574529.0      5
H-MC       XY24       484940.0      4
H-LC       AR         369758.0      2
M-MC       XY24       315812.0      2
H-SC       X40N       295446.0      4
           SR         246329.0      2
           OX40N      245793.0      4
H-LC       X40N       221805.0      4
H-MC       X40        220228.0      4
           XY25       188064.0      2
M-LC       XY24       162257.0      2
M-SC       AR         161451.0      2
L-SC       XR         161168.0      2
M-LC       X5K        122774.0      1
M-SC       OX40N      122364.0      4
           XR         117952.0      2
           XY25       113248.0      1
H-MC       XR         109282.0      1
           X40N       109068.0      2
M-LC       XR         103179.0      1
H-LC       X200        86893.0      1
L-SC       OX40N       84190.0      1
M-SC       X40         71276.0      1
H-LC       XY24        65281.0      1
H-MC       OX40N       64909.0      1
H-SC       MH          62626.0      1
L-MC       XR          59740.0      1
H-LC       XY25        59728.0      2
L-LC       XY25        38655.0      1
H-MC       MH          16266.0      1
M-LC       XY25        10808.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 43.0 seconds
